<a href="https://colab.research.google.com/github/me-nata/NLP-Clusterizacao_e_Classificacao/blob/main/NLP_AS04_NATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\

\

<center>
<h1>Natual Language Processing $-$ AS04</h1>
Trabalho da disciplina de Processamento de Linguagem Natural: vetorizção de texto, exploração de métodos de clusterização e classificação e Utilização do PCA para visualização dos dados

\

\

Natã Oliveira $-$ $712480$

\

<img src="https://drive.google.com/uc?id=1O0hVXmckWjcKCipbGIcqyMSY-PxG8r34">
</center>

\



\

\

In [3]:
!pip install hdbscan

  Using cached hdbscan-0.8.33.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039178 sha256=3440010418b47848a33836b973f443ed9ad46bb1560c765c97b81c77294bef0e
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5


In [4]:
import re
import os
import requests
import time
import random

import numpy as np
import pandas as pd

import spacy

import sklearn
# data
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

# clustering
import sklearn.cluster as cluster
import sklearn.mixture as mixture
from sklearn.decomposition import PCA
import hdbscan

# classfication
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier

# metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score

# plot
import plotly.express as px

# Clustering
\

<center>
<h4>
O objetivo dessa clusterização é explorar métodos variados de classificação de texto vetorizados
</h4>

\

<h2>Dataset<h2>

\

In [5]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
newsgroups_train = fetch_20newsgroups(subset='train', categories=['talk.politics.misc'])
DATASET = newsgroups_train.data
random.choice(DATASET)

'Organization: City University of New York\nFrom: <F36SI@CUNYVM.BITNET>\nSubject: Model United Nations\nLines: 3\n\n    Just observed at the National Model United Nations here in NYC.\n    Just one word on it : AWSOME.\n                                 Peace, matt\n'

\

<h2> Preprocessing <h2>

\

In [7]:
def norm_text(text: str) -> str:
    text = text.lower()
    text = re.sub('\n', ' ', text)
    text = re.sub('(from:|subject:)(.+)(writes:|wrote:|nntp-posting-host:|said:)',
                  '', text, flags=re.DOTALL) # content
    text = re.sub('\'', '', text)
    text = re.sub('[^\w]', ' ', text) # only word
    text = re.sub('\d', ' ', text) # no digit
    text = re.sub('[_]+', ' ', text)
    text = re.sub('\s{2,}', ' ', text)
    return text.strip()

In [8]:
DATASET[0]

'From: steveh@thor.isc-br.com (Steve Hendricks)\nSubject: Re: Limiting Govt (was Re: Employment (was Re: Why not concentrate...)\nSummary: More on failed governments\nOrganization: Failed Libertarian Opportunities, Inc.\nLines: 24\nNntp-Posting-Host: thor.isc-br.com\n\nIn article <18APR199314034390@venus.tamu.edu> gmw0622@venus.tamu.edu (Mr. Grinch) writes:\n>In article <1993Apr18.172531.10946@isc-br.isc-br.com>, steveh@thor.isc-br.com (Steve Hendricks) writes...\n>> \n>:It would seem that a society with a "failed" government would be an ideal\n>:setting for libertarian ideals to be implemented.  Now why do you suppose\n>:that never seems to occur?...\n>\n>\n>I fail to see why you should feel this way in the first place.  Constant\n>combat isn\'t particularly conducive to intellectual theorizing.  Also,\n>they tend to get invaded before they can come to anything like a stable\n>society anyway. \n\nAnd the reason that the Soviet Union couldn\'t achieve the ideal of pure\ncommunism was t

In [9]:
norm_text(DATASET[0])

'in article apr isc br isc br com steveh thor isc br com steve hendricks writes it would seem that a society with a failed government would be an ideal setting for libertarian ideals to be implemented now why do you suppose that never seems to occur i fail to see why you should feel this way in the first place constant combat isnt particularly conducive to intellectual theorizing also they tend to get invaded before they can come to anything like a stable society anyway and the reason that the soviet union couldnt achieve the ideal of pure communism was the hostility of surrounding capitalist nations uh huh somehow this all sounds familiar once again utopian dreams are confronted by the real world mr grinch steve hendricks domain steveh thor isc br com one thing about data it sure does cut uucp uunet isc br thor steveh the bulls t r hofferbert ma bell'

In [10]:
CORPUS = [norm_text(doc) for doc in DATASET]
len(CORPUS)

465

\

<h2>Vetorization<h2>

In [11]:
def plot_clusters(data, alg, args, kwds) -> None:
    start_time = time.time()
    model = alg(*args, **kwds)
    labels = model.fit_predict(data)
    end_time = time.time()

    pca = PCA(n_components=2)
    data_reduced = pca.fit_transform(data)

    df = pd.DataFrame(data_reduced, columns=["X", "Y"])
    df["Cluster"] = labels

    fig = px.scatter(df, x="X", y="Y", color="Cluster", title=f'Clustering com {alg.__name__}')
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.show()

    print(f'Tempo de execução: {end_time - start_time:.2f} segundos')
    try:
        response = pd.DataFrame(np.hstack((data, labels.reshape(-1, 1))), columns=list(data.columns) + ['LABEL'])
    except:
        columns = [f'Feature{i}' for i in range(data.shape[1])] + ['LABEL']
        response = pd.DataFrame(np.hstack((data, labels.reshape(-1, 1))), columns=columns)

    return response

\

## TF-IDF

\

In [12]:
vectorizer = CountVectorizer()
docTermCount = vectorizer.fit_transform(CORPUS)
terms = vectorizer.get_feature_names_out()

transformer = TfidfTransformer()
docTermTfidf = transformer.fit_transform(docTermCount)

matrix = pd.DataFrame(docTermTfidf.A, columns=terms)
matrix

,aa,aaa,aaaaaaaaaaaa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abandoned,abandonment,abbey,...,zion,zionists,zip,zippy,zone,zoo,zooid,zorba,zreh,zumwalt
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
461,0.0,0.0,0.0,0.0,0.069349,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
462,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
463,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


\

<h3>K-means (K=4)<h3>

\

In [13]:
plot_clusters(matrix, cluster.KMeans, (), {'n_clusters':4}).head(3)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Tempo de execução: 2.99 segundos


,aa,aaa,aaaaaaaaaaaa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abandoned,abandonment,abbey,...,zionists,zip,zippy,zone,zoo,zooid,zorba,zreh,zumwalt,LABEL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


\

<h3>Spectral Clustering (K=6)<h3>

\

In [14]:
plot_clusters(matrix, cluster.SpectralClustering, (), {'n_clusters':6}).head(3)

Tempo de execução: 0.56 segundos


,aa,aaa,aaaaaaaaaaaa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abandoned,abandonment,abbey,...,zionists,zip,zippy,zone,zoo,zooid,zorba,zreh,zumwalt,LABEL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


\

<h3>Gaussian Mixture<h3>

\

In [15]:
plot_clusters(matrix, mixture.GaussianMixture, (), {'n_components': 2}).head(3)

Tempo de execução: 685.75 segundos


,aa,aaa,aaaaaaaaaaaa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abandoned,abandonment,abbey,...,zionists,zip,zippy,zone,zoo,zooid,zorba,zreh,zumwalt,LABEL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


\

<h3>Agglomerative Clustering<h3>

\

In [16]:
plot_clusters(matrix, cluster.AgglomerativeClustering, (), {}).head(3)

Tempo de execução: 0.87 segundos


,aa,aaa,aaaaaaaaaaaa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abandoned,abandonment,abbey,...,zionists,zip,zippy,zone,zoo,zooid,zorba,zreh,zumwalt,LABEL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


\

<h3>DBSCAN<h3>

\

In [17]:
plot_clusters(matrix, cluster.DBSCAN, (), {'eps': 1}).head(3)

Tempo de execução: 0.48 segundos


,aa,aaa,aaaaaaaaaaaa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abandoned,abandonment,abbey,...,zionists,zip,zippy,zone,zoo,zooid,zorba,zreh,zumwalt,LABEL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


\

<h3>HDBSCAN<h3>

\

In [18]:
plot_clusters(matrix, hdbscan.HDBSCAN, (), {}).head(3)

Tempo de execução: 5.68 segundos


,aa,aaa,aaaaaaaaaaaa,aammmaaaazzzzzziinnnnggggg,aaron,ab,abandon,abandoned,abandonment,abbey,...,zionists,zip,zippy,zone,zoo,zooid,zorba,zreh,zumwalt,LABEL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


\

\

\

## Word2Vec

\

In [19]:
nlp = spacy.load('en_core_web_sm')
matrix = np.array([nlp(doc).vector for doc in CORPUS])
matrix.shape

(465, 96)

\

<h3>K-means (K=4)<h3>

\

In [20]:
plot_clusters(matrix, cluster.KMeans, (), {'n_clusters':4}).head(3)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Tempo de execução: 0.07 segundos


,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature87,Feature88,Feature89,Feature90,Feature91,Feature92,Feature93,Feature94,Feature95,LABEL
0,-0.046915,-0.477183,0.179054,0.172806,-0.215599,-0.218850,0.316136,0.209764,-0.083612,0.312774,...,-0.105146,-0.415421,0.269308,0.089296,-0.220884,0.458936,0.443160,-0.286755,0.086184,1.0
1,0.107362,-0.258998,-0.084641,0.168722,-0.029830,-0.303736,0.428301,0.034013,0.086021,0.164651,...,0.076758,-0.254271,0.275533,0.196394,-0.226447,0.377280,0.479699,-0.175498,0.132597,3.0
2,0.248880,-0.322145,0.057027,0.088146,0.034683,-0.184076,0.396054,0.189700,0.086001,0.263018,...,-0.043261,-0.435186,0.257973,-0.034907,-0.136715,0.488055,0.555378,-0.148462,0.109137,1.0


\

<h3>Spectral Clustering (K=6)<h3>

\

In [21]:
plot_clusters(matrix, cluster.SpectralClustering, (), {'n_clusters':6}).head(3)

Tempo de execução: 0.13 segundos


,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature87,Feature88,Feature89,Feature90,Feature91,Feature92,Feature93,Feature94,Feature95,LABEL
0,-0.046915,-0.477183,0.179054,0.172806,-0.215599,-0.218850,0.316136,0.209764,-0.083612,0.312774,...,-0.105146,-0.415421,0.269308,0.089296,-0.220884,0.458936,0.443160,-0.286755,0.086184,0.0
1,0.107362,-0.258998,-0.084641,0.168722,-0.029830,-0.303736,0.428301,0.034013,0.086021,0.164651,...,0.076758,-0.254271,0.275533,0.196394,-0.226447,0.377280,0.479699,-0.175498,0.132597,0.0
2,0.248880,-0.322145,0.057027,0.088146,0.034683,-0.184076,0.396054,0.189700,0.086001,0.263018,...,-0.043261,-0.435186,0.257973,-0.034907,-0.136715,0.488055,0.555378,-0.148462,0.109137,0.0


\

<h3>Gaussian Mixture<h3>

\

In [22]:
plot_clusters(matrix, mixture.GaussianMixture, (), {'n_components': 2}).head(3)

Tempo de execução: 0.04 segundos


,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature87,Feature88,Feature89,Feature90,Feature91,Feature92,Feature93,Feature94,Feature95,LABEL
0,-0.046915,-0.477183,0.179054,0.172806,-0.215599,-0.218850,0.316136,0.209764,-0.083612,0.312774,...,-0.105146,-0.415421,0.269308,0.089296,-0.220884,0.458936,0.443160,-0.286755,0.086184,1.0
1,0.107362,-0.258998,-0.084641,0.168722,-0.029830,-0.303736,0.428301,0.034013,0.086021,0.164651,...,0.076758,-0.254271,0.275533,0.196394,-0.226447,0.377280,0.479699,-0.175498,0.132597,0.0
2,0.248880,-0.322145,0.057027,0.088146,0.034683,-0.184076,0.396054,0.189700,0.086001,0.263018,...,-0.043261,-0.435186,0.257973,-0.034907,-0.136715,0.488055,0.555378,-0.148462,0.109137,1.0


\

<h3>Agglomerative Clustering<h3>

\

In [23]:
plot_clusters(matrix, cluster.AgglomerativeClustering, (), {}).head(3)

Tempo de execução: 0.02 segundos


,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature87,Feature88,Feature89,Feature90,Feature91,Feature92,Feature93,Feature94,Feature95,LABEL
0,-0.046915,-0.477183,0.179054,0.172806,-0.215599,-0.218850,0.316136,0.209764,-0.083612,0.312774,...,-0.105146,-0.415421,0.269308,0.089296,-0.220884,0.458936,0.443160,-0.286755,0.086184,0.0
1,0.107362,-0.258998,-0.084641,0.168722,-0.029830,-0.303736,0.428301,0.034013,0.086021,0.164651,...,0.076758,-0.254271,0.275533,0.196394,-0.226447,0.377280,0.479699,-0.175498,0.132597,1.0
2,0.248880,-0.322145,0.057027,0.088146,0.034683,-0.184076,0.396054,0.189700,0.086001,0.263018,...,-0.043261,-0.435186,0.257973,-0.034907,-0.136715,0.488055,0.555378,-0.148462,0.109137,0.0


\

<h3>DBSCAN<h3>

\

In [24]:
plot_clusters(matrix, cluster.DBSCAN, (), {'eps': 1}).head(3)

Tempo de execução: 0.01 segundos


,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature87,Feature88,Feature89,Feature90,Feature91,Feature92,Feature93,Feature94,Feature95,LABEL
0,-0.046915,-0.477183,0.179054,0.172806,-0.215599,-0.218850,0.316136,0.209764,-0.083612,0.312774,...,-0.105146,-0.415421,0.269308,0.089296,-0.220884,0.458936,0.443160,-0.286755,0.086184,0.0
1,0.107362,-0.258998,-0.084641,0.168722,-0.029830,-0.303736,0.428301,0.034013,0.086021,0.164651,...,0.076758,-0.254271,0.275533,0.196394,-0.226447,0.377280,0.479699,-0.175498,0.132597,0.0
2,0.248880,-0.322145,0.057027,0.088146,0.034683,-0.184076,0.396054,0.189700,0.086001,0.263018,...,-0.043261,-0.435186,0.257973,-0.034907,-0.136715,0.488055,0.555378,-0.148462,0.109137,0.0


\

<h3>HDBSCAN<h3>

\

In [25]:
plot_clusters(matrix, hdbscan.HDBSCAN, (), {}).head(3)

Tempo de execução: 0.05 segundos


,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature87,Feature88,Feature89,Feature90,Feature91,Feature92,Feature93,Feature94,Feature95,LABEL
0,-0.046915,-0.477183,0.179054,0.172806,-0.215599,-0.218850,0.316136,0.209764,-0.083612,0.312774,...,-0.105146,-0.415421,0.269308,0.089296,-0.220884,0.458936,0.443160,-0.286755,0.086184,-1.0
1,0.107362,-0.258998,-0.084641,0.168722,-0.029830,-0.303736,0.428301,0.034013,0.086021,0.164651,...,0.076758,-0.254271,0.275533,0.196394,-0.226447,0.377280,0.479699,-0.175498,0.132597,-1.0
2,0.248880,-0.322145,0.057027,0.088146,0.034683,-0.184076,0.396054,0.189700,0.086001,0.263018,...,-0.043261,-0.435186,0.257973,-0.034907,-0.136715,0.488055,0.555378,-0.148462,0.109137,-1.0


\

\

# Classification

\

<center>

<h4>
O objetivo dessa classificar é explorar a base de dados de comentários de uma companhia de viajens aéreas e dizer se um comentário é positivo, neutro ou negativo de acordo com os textos que já foram classificados
</h4>

\

<h2>Dataset<h2>

\

In [26]:
link = 'https://drive.google.com/uc?id=1V4djLAzL19ZcirxL7qP-j5pXh29AlviQ'
df = pd.read_csv(link)[['text', 'airline_sentiment']]
df

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
...,...,...
14635,@AmericanAir thank you we got on a different f...,positive
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative
14637,@AmericanAir Please bring American Airlines to...,neutral
14638,"@AmericanAir you have my money, you change my ...",negative


In [27]:
df['airline_sentiment'].replace({'positive': 1, 'neutral': 0, 'negative': -1}, inplace=True)

\

<h2> Preprocessing <h2>

\

In [28]:
def norm_text(text: str) -> str:
    text = text.lower()
    text = re.sub('@(\w)+', ' ', text) # no @ reference
    text = re.sub('(http|https)(.+)', ' ', text, flags=re.DOTALL) # no link
    text = re.sub('[^\w]', ' ', text) # only word
    text = re.sub('\d', ' ', text) # no digit
    text = re.sub('\s{2,}', ' ', text) # no more than spaces
    return text.strip()

In [29]:
df['text'][10]

'@VirginAmerica did you know that suicide is the second leading cause of death among teens 10-24'

In [30]:
norm_text(df['text'][10])

'did you know that suicide is the second leading cause of death among teens'

In [31]:
df['text'] = df['text'].apply(norm_text)

\

<h2> Apply Method function <h2>

\

In [32]:
def run(data, sentiment) -> None:
    end_time = time.time()

    X_train, X_test, Y_train, Y_test = train_test_split(data, sentiment, shuffle=True)

    mdls = [
        ('Nayve bayes', GaussianNB),
        ('Decision tree', DecisionTreeClassifier),
        ('SVM', SVC),
        ('Logistic regression', LogisticRegression),
        ('Random Forest', RandomForestClassifier)
    ]

    for name, func in mdls:
        print(name, ':')


        mdl = func()
        ovr = OneVsRestClassifier(mdl)
        start_time = time.time()
        ovr.fit(X_train, Y_train)
        end_time = time.time()
        y_pred = ovr.predict(X_test)

        acc = round(accuracy_score(Y_test, y_pred)*100, 2)
        precision, recall, f1score, support = score(Y_test, y_pred, average='micro')
        print(f'accuracy: {acc}')
        print(f'Precision: {precision}')
        print(f'Recall: {recall}')
        print(f'F1-score: {f1score}')

        print(f'\nTIME: {end_time-start_time}')

\

## TF-IDF

\

In [40]:
vectorizer = CountVectorizer()
docTermCount = vectorizer.fit_transform(df['text'])
terms = vectorizer.get_feature_names_out()

transformer = TfidfTransformer()
docTermTfidf = transformer.fit_transform(docTermCount)

matrix = pd.DataFrame(docTermTfidf.A, columns=terms)
matrix

,_exact_,aa,aaaand,aaadvantage,aaalwayslate,aadavantage,aadelay,aadv,aadvantage,aafail,...,zip,zippers,zombie,zone,zones,zoom,zrh,zukes,zurich,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
run(matrix, df['airline_sentiment'])

Nayve bayes :
accuracy: 45.38
Precision: 0.45382513661202184
Recall: 0.45382513661202184
F1-score: 0.45382513661202184

TIME: 7.659223556518555
Decision tree :
accuracy: 62.84
Precision: 0.6284153005464481
Recall: 0.6284153005464481
F1-score: 0.6284153005464481

TIME: 270.79754281044006
SVM :
accuracy: 79.26
Precision: 0.7926229508196722
Recall: 0.7926229508196722
F1-score: 0.7926229508196722

TIME: 2667.0564918518066
Logistic regression :
accuracy: 79.59
Precision: 0.7959016393442623
Recall: 0.7959016393442623
F1-score: 0.7959016393442623

TIME: 47.18198013305664
Random Forest :
accuracy: 75.25
Precision: 0.7524590163934426
Recall: 0.7524590163934426
F1-score: 0.7524590163934426

TIME: 166.3971302509308
